# Machine Translation - English to Hindi

It translates English sentenece into Hindi sentence

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
# Define Parameters

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 2867 # 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'hin-eng/hin.txt'

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [5]:
# Let's see first 10 rows of data
lines[:10]

['Wow!\tवाह!',
 'Help!\tबचाओ!',
 'Jump.\tउछलो.',
 'Jump.\tकूदो.',
 'Jump.\tछलांग.',
 'Hello!\tनमस्ते।',
 'Hello!\tनमस्कार।',
 'Cheers!\tवाह-वाह!',
 'Cheers!\tचियर्स!',
 'Got it?\tसमझे कि नहीं?']

In [6]:
# input_texts - collection of english lines
# target_texts - collection of hindi lines
# input_characters - vocabulary of english characters
# target_characters - vocabulary of hindi characters

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
# num_encoder_tokens - size of eng vocab
# num_decoder_tokens - size of hin vocab
# max_encoder_seq_length - Longest sequence length [Eng]
# max_decoder_seq_length - Longest sequence length [Hindi]

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
print('Number of samples/input examples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples/input examples: 2867
Number of unique input tokens: 70
Number of unique output tokens: 92
Max sequence length for inputs: 124
Max sequence length for outputs: 123


In [9]:
# Create tuple of (index, character)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [10]:
# Declare array using format (m, vocab_size, maximum_sentence_length)
# Where m - No. of input examples
# max_encoder_seq_length - vocab_size of eng language
# max_decoder_seq_length - vocab_size of hindi language
# num_decoder_tokens - maximum sentence of output length

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [11]:
# Iterate over input text and target text
# Fill encoder_input_data with (index_of_input_text, index_of_char, index_of_char_index_dict)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # num_encoder_tokens - vocab size
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens)) # num_decoder_tokens - vocab size
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 2293 samples, validate on 574 samples
Epoch 1/100
2293/2293 [==============================] - 45s 20ms/step - loss: 0.8216 - val_loss: 1.3057
Epoch 2/100
2293/2293 [==============================] - 46s 20ms/step - loss: 0.7564 - val_loss: 1.3009
Epoch 3/100
2293/2293 [==============================] - 49s 21ms/step - loss: 0.7185 - val_loss: 1.1803
Epoch 4/100
2293/2293 [==============================] - 51s 22ms/step - loss: 0.6552 - val_loss: 1.0912
Epoch 5/100
2293/2293 [==============================] - 52s 23ms/step - loss: 0.6073 - val_loss: 1.0095
Epoch 6/100
2293/2293 [==============================] - 52s 23ms/step - loss: 0.5719 - val_loss: 0.9681
Epoch 7/100
2293/2293 [==============================] - 52s 23ms/step - loss: 0.5409 - val_loss: 0.9499
Epoch 8/100
2293/2293 [==============================] - 53s 23ms/step - loss: 0.5196 - val_loss: 0.9238
Epoch 9/100
2293/2293 [==============================] - 53s 23ms/step - loss: 0.5017 - val_loss: 0.8818
Epoch 10

Epoch 78/100
2293/2293 [==============================] - 57s 25ms/step - loss: 0.1617 - val_loss: 1.0056
Epoch 79/100
2293/2293 [==============================] - 56s 25ms/step - loss: 0.1590 - val_loss: 1.0183
Epoch 80/100
2293/2293 [==============================] - 56s 24ms/step - loss: 0.1564 - val_loss: 1.0277
Epoch 81/100
2293/2293 [==============================] - 57s 25ms/step - loss: 0.1538 - val_loss: 1.0275
Epoch 82/100
2293/2293 [==============================] - 56s 25ms/step - loss: 0.1523 - val_loss: 1.0225
Epoch 83/100
2293/2293 [==============================] - 57s 25ms/step - loss: 0.1497 - val_loss: 1.0550
Epoch 84/100
2293/2293 [==============================] - 57s 25ms/step - loss: 0.1478 - val_loss: 1.0578
Epoch 85/100
2293/2293 [==============================] - 57s 25ms/step - loss: 0.1461 - val_loss: 1.0638
Epoch 86/100
2293/2293 [==============================] - 81s 35ms/step - loss: 0.1440 - val_loss: 1.0599
Epoch 87/100
2293/2293 [======================

In [16]:
# Save model
model.save('s2s.h5')

C:\Users\surinder.kumar01\AppData\Local\conda\conda\envs\tia\lib\site-packages\keras-2.1.6-py3.6.egg\keras\engine\topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [17]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [18]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [19]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [30]:
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: तुम्हारे पास कितनी किताबें हैं?

-
Input sentence: Help!
Decoded sentence: उसने मुझे मत में दोना में लग्या करा।

-
Input sentence: Jump.
Decoded sentence: तुम्हे इस तरह की आदत तोड़ दिना चाहिए।

-
Input sentence: Jump.
Decoded sentence: तुम्हे इस तरह की आदत तोड़ दिना चाहिए।

-
Input sentence: Jump.
Decoded sentence: तुम्हे इस तरह की आदत तोड़ दिना चाहिए।

